# **Download Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [ ]:
%cd /content/drive/MyDrive/DL/

In [ ]:
!kaggle datasets download -d pratik2901/multiclass-weather-dataset --unzip

# **Import**

In [ ]:
import cv2
import numpy as np
import numpy
from matplotlib import pyplot
from PIL import Image 
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Dropout, Input, MaxPooling2D, AveragePooling2D, Activation, add
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score

# **Ready Dataset**

In [ ]:

images = []
labels = []
resize_images = []
convert = {'Cloudy':0, 'Rain':1, 'Shine':2, 'Sunrise':3}

path = "/content/drive/MyDrive/DL/Multi-class Weather Dataset"
print(os.listdir(path))
for filename in os.listdir(path):
  for filename1 in os.listdir(path+'/'+filename):
      img = cv2.imread(path+'/'+filename+'/'+filename1)
      if img is not None:
          images.append(img)
          labels.append(convert[filename])

In [ ]:
# Data distribution
num_distr = np.zeros(4, dtype=int)
labels_name = ['Cloudy', 'Rain', 'Shine', 'Sunrise']
for i in range(len(labels)):
  num_distr[labels[i]]+=1

print(f'Number of data {len(labels)}')

for i in range(len(labels_name)):
  print(f'{num_distr[i]} of the datasets are {labels_name[i]}')



In [ ]:
#convert images to PLI image and resize images
for i in range(len(images)):
  img = Image.fromarray(images[i], "RGB")
  images[i] = img.resize((80, 80))

In [ ]:
#convert back images from PLI to np array
for i in range(len(images)):
  img = np.array(images[i])
  images[i] = img
images = np.array(images)

In [ ]:
#convert lables to hot encoding
encoder = LabelEncoder()
encoder.fit(labels)
encoded_y = encoder.transform(labels)
ll = np_utils.to_categorical(encoded_y)


# **Split Data**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, ll, test_size=0.2, random_state=30)


# **Build Model**

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001) 

model = Sequential()
model.add(Input(shape = images[0].shape))
model.add(Conv2D(6, padding="valid", strides=1,  dilation_rate=1, kernel_size=5, activation='tanh'))
model.add(AveragePooling2D(pool_size=2, strides=2))
model.add(Conv2D(8, padding="valid", strides=1,  dilation_rate=1, kernel_size=5, activation='tanh'))
model.add(AveragePooling2D(pool_size=2, strides=2))
#model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
#model.add(Dense(8, activation='elu'))
#model.add(Dropout(0.3))
model.add(Dense(8, activation='elu'))
#model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))
model.compile(loss="categorical_crossentropy",
              optimizer = opt,
              metrics=['accuracy'])
model.summary()


# **Build ResNet Model**

In [ ]:
from keras.models import Model

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
def residual_module(layer_in, n_filters):

	merge_input = layer_in
	# check if the number of filters needs to be increase

	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, 1, padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	conv1 = Conv2D(n_filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	conv2 = Conv2D(n_filters, 3, padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	layer_out = add([conv2, merge_input])
	layer_out = Activation('relu')(layer_out)
	return layer_out
input_layer = Input(shape = images[0].shape)
layer = Conv2D(5, padding="valid", strides=1,  dilation_rate=2, kernel_size=3, activation='tanh')(input_layer)
layer = MaxPooling2D(pool_size=2, strides=2)(layer)
layer = residual_module(layer, 16)
layer = residual_module(layer, 16)
layer = residual_module(layer, 32)
layer = residual_module(layer, 32)
layer = Flatten()(layer)

layer = Dense(4, activation='softmax')(layer)

model = Model(inputs=input_layer, outputs=layer)

model.compile(loss="categorical_crossentropy",
              optimizer = opt,
              metrics=['accuracy'])
plot_model(model)

# **Build Inception Model**

In [ ]:
from keras.models import Model
opt = tf.keras.optimizers.Adam(learning_rate=0.001) 

def inception_module(layer_in, f1, f2, f3):
	conv1 = Conv2D(f1, kernel_size=5, padding='same', activation='relu')(layer_in)
	conv3 = Conv2D(f2, kernel_size=5, padding='same', activation='relu')(layer_in)
	conv5 = Conv2D(f3, kernel_size=5, padding='same', activation='relu')(layer_in)
	pool = MaxPooling2D(pool_size=3, strides=1, padding='same')(layer_in)
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

input_layer = Input(shape = images[0].shape)
layer = Conv2D(5, padding="valid", strides=1,  dilation_rate=1, kernel_size=3, activation='tanh')(input_layer)
layer = MaxPooling2D(pool_size=2, strides=2)(layer)
layer = Conv2D(5, padding="valid", strides=1,  dilation_rate=1, kernel_size=3, activation='tanh')(layer)
layer = MaxPooling2D(pool_size=2, strides=2)(layer)
layer = Conv2D(5, padding="valid", strides=1,  dilation_rate=1, kernel_size=3, activation='tanh')(layer)
layer = MaxPooling2D(pool_size=2, strides=2)(layer)
layer = inception_module(layer,  16, 16, 16)
#layer = inception_module(layer,  16, 32, 16)
#layer = inception_module(layer,  16, 16, 16)
layer = Flatten()(layer)
layer = Dense(4, activation='softmax')(layer)
model = Model(inputs=input_layer, outputs=layer)

model.compile(loss="categorical_crossentropy",
              optimizer = opt,
              metrics=['accuracy'])
#plot_model(model)



# **Train Model**

In [ ]:
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

# Evaluate train and test set 

In [ ]:
loss, accuracy = model.evaluate(x_train, y_train)
print('Accuracy of train set: %.2f' % (accuracy*100))
print('Loss of train set: %.3f' % (loss))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Accuracy of test set: %.2f' % (accuracy*100))
print('Loss of test set: %.3f' % (loss))

# Plot

*Plot* loss and accuracy of train set

In [ ]:
history = history.history

In [ ]:
plt.plot(history['accuracy'])
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.show()
plt.plot(history['loss'], 'green')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()

*Plot* loss and accuracy of validation set

In [ ]:
plt.plot(history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.show()
plt.plot(history['val_loss'], 'green')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()

comparing loss of train and validation set

In [ ]:
#plt.figure(figsize=(12, 20))

plt.plot(history['loss'], color='blue')
plt.plot(history['val_loss'], color='green')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val.'])
plt.show()

comparing accuracy of train and validation set

In [ ]:

plt.plot(history['accuracy'], color='blue')
plt.plot(history['val_accuracy'], color='green')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val.'])
plt.show()

# **k fold cross validation**

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
cvscores = []
scores = []
print(kfold.split(images, ll))
for train, test in kfold.split(images, ll):
  model.fit(images[train], ll[train], epochs=50)
  score = model.evaluate(images[test], ll[test], verbose=0)
  scores.append(score)
  cvscores.append(score * 100)
  print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

#print loss and accuracy for each training
for i in range(len(cvscores)):
  print("%s: %.3f" % (model.metrics_names[0], scores[i][0]))
  print("%s: %.2f%%" % (model.metrics_names[1], scores[i][1]*100))

  

In [ ]:
#find the predicted result
y_predict = model.predict(x_test)
y_predict_int = np.argmax(y_predict, axis=1)
#convert back from hot encoding for comparing with predicted result
y_true = np.argmax(y_test, axis = 1)

print(classification_report(y_true, y_predict_int))